### 简单动态规划（超时）
* 第一层动态规划（最多一次交易）：
    * dp[i] = max(dp[i-1], dp[i] - dp[j])，0<=j<=i
        * 第 i 天的最大收益 = max（第 i-1 天的最大收益，第 j 天（遍历）买入第 i 天卖出的最大收益）
        * 若为第一种情况，表示未进行交易
        * 若为第二种情况，表示进行交易且收益大于前一天的最大收益
* 第二层动态对话（最多两次交易）：
    * 第二层动态规划需要在第一层动态规划的基础上进行，即上一层动态规划的值为 dp[1][:]（dp[0][:]=0，为占位符，方便计算）
    * 初始化，dp[2][i] = max(dp[2][i-1], prices[i]-prices[0])
        * 第一种情况，表示并未进行交易，初始最大值为前一天的最大值
        * 第二种情况，表示第 0 天买入，第 i 天卖出的收益，且大于前一天的最大收益
    * 遍历第 j 天买入第 i 天卖出的收益，进一步确定 dp[2][i] 的最大值，dp[2][i] = max(dp[2][i], dp[1][j-1]+prices[i]-prices[j])
        * 第一种情况，表示未进行交易
        * 第二种情况，表示第 j 天买入第 i 天卖出的收益且加上第一层动态规划的第 j-1 天的最大收益（要求第二次交易必须在第一次交易之后）

In [57]:
class Solution:
    def maxProfit(self, prices):
        if not prices:
            return 0
        n = len(prices)
        dp = [[0]*n for _ in range(3)]
        for k in range(1, 3):
            for i in range(1, n):
                # 第一层动态对话，初始化第二层动态规划
                dp[k][i] = max(dp[k][i-1], prices[i]-prices[0])  
                for j in range(1, i+1):
                    # 第二层动态规划
                    dp[k][i] = max(dp[k][i], dp[k-1][j-1]+prices[i]-prices[j])
        return dp[-1][-1]

In [58]:
test = Solution()
prices_li = [[3,3,5,0,0,3,1,4], [1,2,3,4,5]]
for prices in prices_li:
    print(test.maxProfit(prices))

6
4


### 优化上一种方案（超时）
* 先求 dp[k-1][j-1]-prices[j] 最大值, 再求 prices[i] 的最大值

（意义不大，但有助于下一个方案的理解）

In [61]:
class Solution:
    def maxProfit(self, prices):
        if not prices: return 0
        n = len(prices)
        dp = [[0] * n for _ in range(3)]
        for k in range(1, 3):
            for i in range(1, n):
                pre_max = -prices[0]
                for j in range(1, i + 1):
                    pre_max = max(pre_max, dp[k - 1][j - 1] - prices[j])
                dp[k][i] = max(dp[k][i - 1], prices[i] + pre_max)
        return dp[-1][-1]

In [62]:
test = Solution()
prices_li = [[3,3,5,0,0,3,1,4], [1,2,3,4,5]]
for prices in prices_li:
    print(test.maxProfit(prices))

6
4


### 致幻方案

#### 观察上一方案中的 i 循环和 j 循环：
* 本质上更新 dp[k][i]，而每次寻找 pre_max 最大值都是遍历区间 [1, i]，因此 j 循环与 i 循环是重叠的，可以同步更新 pre_max 和 dp[k][i]。

（该方案可通过所有测试样例，但不好理解（大佬除外））

In [87]:
class Solution:
    def maxProfit(self, prices):
        if not prices: return 0
        n = len(prices)
        dp = [[0] * n for _ in range(3)]
        for k in range(1, 3):
            pre_max = -prices[0]             
            for i in range(1, n):                                   
                pre_max = max(pre_max, dp[k - 1][i - 1] - prices[i])    
                dp[k][i] = max(dp[k][i - 1], prices[i] + pre_max)       
        return dp[-1][-1]

In [88]:
test = Solution()
prices_li = [[3,3,5,0,0,3,1,4], [1,2,3,4,5]]
for prices in prices_li:
    print(test.maxProfit(prices))

6
4


### 优化可读性方案

#### 最多一次交易的动态规划：
* dp[i] = max(dp[i-1], dp[i] - dp[j])，0<=j<=i
* 第 i 天的最大收益 = max（第 i-1 天的收益，第 j 天买入第 i 天卖出的收益）
    * 若为第一种情况，表示未进行交易
    * 若为第二种情况，表示进行交易且收益大于前一天的最大收益；dp[j] 为第 i 天之前的最低股票值。
    
#### 最多两次交易的动态规划：
* dp[t][i] = max(dp[t][i-1], prices[i]-min_p)
* 第 t 次交易第 i 天的最大收益 = max（第 t 次交易第 i-1 天的收益，第 i 天卖出的股票收益-第 i 天之前的最小负收益）
* 考虑最多一次交易的动态规划中 dp[j] 为第 i 天之前买入股票的最低值，即最小值，则 min_p 也为第 i 天之前的最小值，但要加入之前交易次数的影响。由于买入股票为负收益，而第 i-1 天的上次交易后的最大收益为 dp[t-1][i-1]，因此最小值 min(min_p, prices[i]-dp[t-1][i-1])。
* 由于 min_p 为第 i 天之前的最小值，则可通过一次遍历，同步计算 min_p 和 dp[t][i]。

In [89]:
class Solution:
    def maxProfit(self, prices):
        if not prices:
            return 0
        n = len(prices)
        dp = [[0]*n for _ in range(3)]
        for t in range(1, 3):
            min_p = prices[0]              # 初始化最小负收益，即买入初始股票
            for i in range(1, n):
                min_p = min(min_p, prices[i]-dp[t-1][i-1])     # 更新最小负收益 
                dp[t][i] = max(dp[t][i-1], prices[i]-min_p)    # 更新当前交易次数下第 i 天的最大收益
        return dp[-1][-1]

In [90]:
test = Solution()
prices_li = [[3,3,5,0,0,3,1,4], [1,2,3,4,5]]
for prices in prices_li:
    print(test.maxProfit(prices))

6
4
